In [1]:
import rdflib 
import pandas as pd 
import numpy as np 
from rdflib import Graph, Literal, RDF, URIRef
from rdflib.namespace import FOAF, XSD, RDF
import math

In [2]:
rst = pd.read_excel('Restaurants_final.xlsx',index_col=[0])
poi = pd.read_excel('POI_final.xlsx',index_col=[0])
pa = pd.read_excel('PA_final.xlsx',index_col=[0])
pa19 = pd.read_excel('PA19_final.xlsx',index_col=[0])
cul = pd.read_excel('Culture_final.xlsx',index_col=[0])
sw = pd.read_excel('SD_final.xlsx')

In [3]:
nb = pd.read_excel('Neighbourhood_crimeIndex.xlsx',index_col=[0] )
nb = nb[['Neighbourhood', 'Hood_ID','crimeIndex']]
nb.head()

,Neighbourhood,Hood_ID,crimeIndex
0,Yonge-St.Clair,97,0.251860
1,York University Heights,27,4.259196
2,Lansing-Westgate,38,0.831561
3,Yorkdale-Glen Park,31,2.827610
4,Stonegate-Queensway,16,0.781926


In [5]:
####initilize name 
rst['idx'] = ['rst'+ '%d'%(i) for i in range(len(rst))]
rst['nb_idx'] = ['nb'+ str(i) for i in rst['hood_number']]
rst = rst.set_index('idx')

poi['idx'] = ['poi'+ '%d'%(i) for i in range(len(poi))]
poi['nb_idx'] = ['nb'+ str(i) for i in poi['hood_number']]
poi = poi.set_index('idx')

pa['idx'] = ['pa'+ '%d'%(i) for i in range(len(pa))]
pa['nb_idx'] = ['nb'+ str(i) for i in pa['hood_number']]
pa = pa.set_index('idx')

pa19['idx'] = ['pa19_'+ '%d'%(i) for i in range(len(pa19))]
pa19['nb_idx'] = ['nb'+ str(i) for i in pa19['hood_number']]
pa19 = pa19.set_index('idx')

cul['idx'] = ['cul'+ '%d'%(i) for i in range(len(cul))]
cul['nb_idx'] = ['nb'+ str(i) for i in cul['hood_number']]
cul = cul.set_index('idx')

station_name = sw['Station']
sw['idx'] = ['st'+ '%d'%(i) for i in range(len(sw))]
sw['nb_idx'] = ['nb'+ str(int(i)) for i in sw['hood_number']]
sw = sw.set_index('idx')

nb['idx'] = ['nb'+ str(i) for i in nb['Hood_ID'] ]
nb = nb.set_index('idx')

In [6]:
#initalize a knowledge graph 
g = Graph()

#the ontology we created
fsro = rdflib.Namespace('http://www.semanticweb.org/abdel/ontologies/2021/10/ProjectOntology/#')
schema = rdflib.Namespace('http://www.schema.org/#')
#bind prefixes
g.bind("fsro", fsro)
g.bind("schema", schema)
g.bind("foaf", FOAF)
g.bind("rdf", RDF)

In [7]:
#### neighbourhood 
nb_list = nb.index.tolist()
nb_ter_list = []
for i in nb.index:
    for j in range(len(nb)): 
        g.add((fsro[i], fsro.hoodName, Literal("{}".format(nb.iloc[j,nb.columns.get_loc('Neighbourhood')]))))  ###hood name 
        g.add((fsro[i], fsro.crimeRating, Literal("{}".format(nb.iloc[j,nb.columns.get_loc('crimeIndex')])) ))  ###hood number
        nb_ter_list.append(fsro[i])


In [8]:
# subway line name
sl_ter_list = []
g.add((fsro.sl1, fsro.lineNumber, Literal("{}".format(1)) )) 
sl_ter_list.append(fsro.sl1)
g.add((fsro.sl2, fsro.lineNumber, Literal("{}".format(2)) )) 
sl_ter_list.append(fsro.sl2)
g.add((fsro.sl3, fsro.lineNumber, Literal("{}".format(3)) )) 
sl_ter_list.append(fsro.sl3)
g.add((fsro.sl4, fsro.lineNumber, Literal("{}".format(4)) )) 
sl_ter_list.append(fsro.sl4)


In [9]:
#### subway station
sw_list = sw.index.tolist()
sw_ter_list = []  
for i in sw.index:
    for j in range(len(sw)):
        g.add((fsro[i], fsro.streetAddress, Literal("{}".format(sw.iloc[j,sw.columns.get_loc('Address')])) ))  ###address
        g.add((fsro[i], fsro.fsa, Literal("{}".format(sw.iloc[j,sw.columns.get_loc('FSR')])) ))   ###fsr
        nb_idx = [i for i, element in enumerate(nb_list) if element == sw.iloc[j,sw.columns.get_loc('nb_idx')]]
        g.add((fsro[i], fsro.isInNeighbourhood, nb_ter_list[nb_idx[0]]))   ## in neibourhood
        g.add((fsro[i], fsro.stopName, Literal("{}".format(sw.iloc[j,sw.columns.get_loc('Station')])) ))   #####stop name
       
        #### which line the station operates 
        if sw.iloc[j,1] == 1:
            g.add((fsro[i], fsro.lineOperating, fsro.sl1))
            g.add((fsro.sl1, fsro.hasStop,fsro[i]))
        if sw.iloc[j,2] == 1:
            g.add((fsro[i], fsro.lineOperating, fsro.sl2))
            g.add((fsro.sl2, fsro.hasStop, fsro[i]))
        if sw.iloc[j,3] == 1:
            g.add((fsro[i], fsro.lineOperating, fsro.sl3))
            g.add((fsro.sl3, fsro.hasStop,fsro[i]))
        if sw.iloc[j,4] == 1:
            g.add((fsro[i], fsro.lineOperating, fsro.sl4))
            g.add((fsro.sl4, fsro.hasStop,fsro[i]))
    sw_ter_list.append(fsro[i])


In [ ]:
#station index 
st_fst = rst.columns.get_loc("Bathurst Station")
st_last = rst.columns.get_loc('Yorkdale Station')
st_list = rst.columns[st_fst:st_last].tolist()

### restaurants
for i in rst.index:
    for j in range (len(rst)):
        g.add((fsro[i], RDF.type, fsro.Restaurant))  ###type 
        g.add((fsro[i], schema.streetAddress, Literal("{}".format(rst.iloc[j,1])) ))  ##Address
        g.add((fsro[i], schema.name, Literal("{}".format(rst.iloc[j,2])) ))  ## Official Name
        g.add((fsro[i], schema.openingHoursSpecification, Literal("{}".format(rst.iloc[j,rst.columns.get_loc('Operation Hours')])) ))  ### operating hour

        ### Subway station within 5km
        for k in range(len(st_list)):
            if rst.iloc[j,st_fst+k] == 1:
                g.add((fsro[i], fsro.nearestSubway, sw_ter_list[k]))
        
        g.add((fsro[i], fsro.fsa, Literal("{}".format(rst.iloc[j,rst.columns.get_loc('FSR')])) ))   ###FSR        
        nb_idx = [i for i, element in enumerate(nb_list) if element == rst.iloc[j,rst.columns.get_loc('nb_idx')]]
        g.add((fsro[i], fsro.isInNeighbourhood, nb_ter_list[nb_idx[0]]))   ## in neibourhood
        g.add((fsro[i], fsro.crimeRating, Literal("{}".format(rst.iloc[j,rst.columns.get_loc('Crime_Rating')])) ))   ###crime rating 
        g.add((fsro[i], fsro.postalCode, Literal("{}".format(rst.iloc[j,rst.columns.get_loc('Postal_Code')])) ))   ###postal code
        
        
        g.add((fsro[i], schema.servesCuisine, Literal("{}".format(rst.iloc[j,0])) ))  ## servesCuisine i.e cuisine type
        g.add((fsro[i], fsro.starRating, Literal("{}".format(rst.iloc[j,rst.columns.get_loc('Rating')])) ))  ##yelp star rating 
        g.add((fsro[i], schema.acceptsReservations, Literal("{}".format(rst.iloc[j,rst.columns.get_loc('Accepts_Reservation')])) )) #Accepts REservation
        g.add((fsro[i], schema.telephone, Literal("{}".format(rst.iloc[j,rst.columns.get_loc('Restaurant Phone')])) )) #Accepts REservation        



In [11]:
####Point of Interest is attraction

#### find index of subway station
st_fst = poi.columns.get_loc("Bathurst Station")
st_last = poi.columns.get_loc('Yorkdale Station')
st_list = poi.columns[st_fst:st_last].tolist()



for i in range(len(poi)):
    for j in range (len(poi)):
        g.add((fsro[i], RDF.type, fsro.Attraction))     ### Class definition
        g.add((fsro[i], schema.streetAddress, Literal("{}".format(poi.iloc[j,poi.columns.get_loc('ADDRESS_FULL')]))))  ##Address
        g.add((fsro[i], schema.name, Literal("{}".format(poi.iloc[j,poi.columns.get_loc('NAME')])) ))  ## Official Name
        g.add((fsro[i], schema.openingHoursSpecification, Literal("{}".format(poi.iloc[j,poi.columns.get_loc('Operation Hours')])) ))  ### operating hour
        
        
        ### Subway station within 5km
        for k in range(len(st_list)):
            if poi.iloc[j,st_fst+k] == 1:
                g.add((fsro[i], fsro.nearestSubway, sw_ter_list[k]))
        
        g.add((fsro[i], fsro.fsa, Literal("{}".format(poi.iloc[j,poi.columns.get_loc('FSR')])) ))   ###FSR        
        nb_idx = [i for i, element in enumerate(nb_list) if element == poi.iloc[j,poi.columns.get_loc('nb_idx')]]
        g.add((fsro[i], fsro.isInNeighbourhood, nb_ter_list[nb_idx[0]]))   ## in neibourhood
        g.add((fsro[i], fsro.crimeRating, Literal("{}".format(poi.iloc[j,poi.columns.get_loc('Crime_Rating')])) ))   ###crime rating 
        g.add((fsro[i], fsro.postalCode, Literal("{}".format(poi.iloc[j,poi.columns.get_loc('Postal_Code')])) ))  ## postal code
        
        
        g.add((fsro[i], fsro.purpose, Literal("{}".format(poi.iloc[j,poi.columns.get_loc('CATEGORY')])) ))  ## purpose of attration
        g.add((fsro[i], fsro.description, Literal("{}".format(poi.iloc[j,poi.columns.get_loc('ATTRACTION_DESC')])) ))  ##description 
        g.add((fsro[i], fsro.telephone, Literal("{}".format(poi.iloc[j,poi.columns.get_loc('PHONE')])) )) #Accepts REservation


In [ ]:
####Making Space for Culture is attraction


#### find index of subway station
st_fst = cul.columns.get_loc("Bathurst Station")
st_last = cul.columns.get_loc('Yorkdale Station')
st_list = cul.columns[st_fst:st_last].tolist()


for i in range(len(cul)):
    for j in range (len(cul)):
        g.add((fsro[i], RDF.type, fsro.Attraction))     ### Class definition
        g.add((fsro[i], schema.streetAddress, Literal("{}".format(cul.iloc[j,cul.columns.get_loc('Full Address')]))))  ##Address
        g.add((fsro[i], schema.name, Literal("{}".format(cul.iloc[j,cul.columns.get_loc('FACILITY NAME')])) ))  ## Official Name
        g.add((fsro[i], schema.openingHoursSpecification, Literal("{}".format(cul.iloc[j,cul.columns.get_loc('Operation Hours')])) ))  ### operating hour
        
        
        ### Subway station within 5km
        for k in range(len(st_list)):
            if cul.iloc[j,st_fst+k] == 1:
                g.add((fsro[i], fsro.nearestSubway, sw_ter_list[k]))
        
        g.add((fsro[i], fsro.fsa, Literal("{}".format(cul.iloc[j,cul.columns.get_loc('FSR')])) ))   ###FSR        
        nb_idx = [i for i, element in enumerate(nb_list) if element == cul.iloc[j,cul.columns.get_loc('nb_idx')]]
        g.add((fsro[i], fsro.isInNeighbourhood, nb_ter_list[nb_idx[0]]))   ## in neibourhood
        g.add((fsro[i], fsro.crimeRating, Literal("{}".format(cul.iloc[j,cul.columns.get_loc('Crime_Rating')])) ))   ###crime rating 
        g.add((fsro[i], fsro.postalCode, Literal("{}".format(cul.iloc[j,cul.columns.get_loc('Postal_Code')])) ))  ## postal code
        
        #### purpose of attraction
        if cul.iloc[j,cul.columns.get_loc('Multipurpose')] == 1:
            g.add((fsro[i], fsro.purpose, Literal('Multipurpose')))
            
        elif cul.iloc[j,cul.columns.get_loc('Performance')] == 1:
            g.add((fsro[i], fsro.purpose, Literal('Performance')))
        
        elif cul.iloc[j,cul.columns.get_loc('Exhibition / Visual Arts')] == 1:
            g.add((fsro[i], fsro.purpose, Literal('Exhibition / Visual Arts')))
        
        elif cul.iloc[j,cul.columns.get_loc('Screen Based')] == 1:
            g.add((fsro[i], fsro.purpose, Literal('Screen Based')))
            
        elif cul.iloc[j,cul.columns.get_loc('Library')] == 1:
            g.add((fsro[i], fsro.purpose, Literal('Library')))
            
            
        
        g.add((fsro[i], fsro.description, Literal("{}".format(cul.iloc[j,cul.columns.get_loc('Desc')])) ))  ##description 


In [ ]:
##### Public Art is Artinstallation


#### find index of subway station
st_fst = pa.columns.get_loc("Bathurst Station")
st_last = pa.columns.get_loc('Yorkdale Station')
st_list = pa.columns[st_fst:st_last].tolist()


for i in range(len(cul)):
    for j in range (len(cul)):
        g.add((fsro[i], RDF.type, fsro.ArtInstallation))     ### Class definition
        g.add((fsro[i], schema.streetAddress, Literal("{}".format(pa.iloc[j,pa.columns.get_loc('Full_Address')]))))  ##Address
        g.add((fsro[i], schema.name, Literal("{}".format(pa.iloc[j,pa.columns.get_loc('Title')])) ))  ## Official Name
        g.add((fsro[i], schema.openingHoursSpecification, Literal("{}".format(pa.iloc[j,pa.columns.get_loc('Operation Hours')])) ))  ### operating hour
        
        
        ### Subway station within 5km
        for k in range(len(st_list)):
            if pa.iloc[j,st_fst+k] == 1:
                g.add((fsro[i], fsro.nearestSubway, sw_ter_list[k]))
        
        g.add((fsro[i], fsro.fsa, Literal("{}".format(pa.iloc[j,pa.columns.get_loc('FSR')])) ))   ###FSR        
        nb_idx = [i for i, element in enumerate(nb_list) if element == pa.iloc[j,pa.columns.get_loc('nb_idx')]]
        g.add((fsro[i], fsro.isInNeighbourhood, nb_ter_list[nb_idx[0]]))   ## in neibourhood
        g.add((fsro[i], fsro.crimeRating, Literal("{}".format(pa.iloc[j,pa.columns.get_loc('Crime_Rating')])) ))   ###crime rating 
        g.add((fsro[i], fsro.postalCode, Literal("{}".format(pa.iloc[j,pa.columns.get_loc('Postal_Code')])) ))  ## postal code
 
               
        g.add((fsro[i], fsro.hasArtist, Literal("{}".format(pa.iloc[j,pa.columns.get_loc('Artist')])) ))  ##has artist 
        g.add((fsro[i], fsro.hasPlacement, Literal("{}".format(pa.iloc[j,pa.columns.get_loc('Placement')])) ))  ###placement

In [ ]:
##### Public Art work asset 2019 is Artinstallation


#### find index of subway station
st_fst = pa19.columns.get_loc("Bathurst Station")
st_last = pa19.columns.get_loc('Yorkdale Station')
st_list = pa19.columns[st_fst:st_last].tolist()


for i in range(len(cul)):
    for j in range (len(cul)):
        g.add((fsro[i], RDF.type, fsro.ArtInstallation))     ### Class definition
        g.add((fsro[i], schema.streetAddress, Literal("{}".format(pa19.iloc[j,pa19.columns.get_loc('ADDRESS')]))))  ##Address
        g.add((fsro[i], schema.name, Literal("{}".format(pa19.iloc[j,pa19.columns.get_loc('ARTWORKNAM')])) ))  ## Official Name
        g.add((fsro[i], schema.openingHoursSpecification, Literal("{}".format(pa19.iloc[j,pa19.columns.get_loc('Operation Hours')])) ))  ### operating hour
        
        
        ### Subway station within 5km
        for k in range(len(st_list)):
            if pa19.iloc[j,st_fst+k] == 1:
                g.add((fsro[i], fsro.nearestSubway, sw_ter_list[k]))
        
        g.add((fsro[i], fsro.fsa, Literal("{}".format(pa19.iloc[j,pa19.columns.get_loc('FSR')])) ))   ###FSR        
        nb_idx = [i for i, element in enumerate(nb_list) if element == pa19.iloc[j,pa19.columns.get_loc('nb_idx')]]
        g.add((fsro[i], fsro.isInNeighbourhood, nb_ter_list[nb_idx[0]]))   ## in neibourhood
        g.add((fsro[i], fsro.crimeRating, Literal("{}".format(pa19.iloc[j,pa19.columns.get_loc('Crime_Rating')])) ))   ###crime rating 
        g.add((fsro[i], fsro.postalCode, Literal("{}".format(pa19.iloc[j,pa19.columns.get_loc('Postal_Code')])) ))  ## postal code
 
               
        g.add((fsro[i], fsro.hasArtist, Literal("{}".format(pa19.iloc[j,pa19.columns.get_loc('ARTIST')])) ))  ##has artist 
        g.add((fsro[i], fsro.hasPlacement, Literal("{}".format(pa19.iloc[j,pa19.columns.get_loc('INDOORS_OU')])) ))  ###placement

In [ ]:
for s,p,o in g:
    print((s,p,o))
    print('\n')

In [ ]:
g.serialize(destination="project_ttl.ttl")